# An implementation of sequence to sequence learning for performing addition

Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be reversed, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits reversed:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits reversed:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits reversed:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits reversed:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs

In [ ]:
import numpy as np
from six.moves import range
import sys

In [ ]:
import keras as K
import tensorflow as tf
from keras.models import Sequential
from keras import layers

# config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
config = tf.ConfigProto(device_count = {'GPU': 1 , 'CPU': 8} ) 
config.gpu_options.allow_growth = True

sess = tf.Session(config=config) 
K.backend.set_session(sess)

In [ ]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.

        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.

        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [ ]:
# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

In [ ]:
ctable.encode(C='0 +', num_rows=10)

In [ ]:
x = ctable.encode(C='0 +', num_rows=5)
ctable.decode(x)

In [ ]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [ ]:
def generate_training(TRAINING_SIZE = 50000, DIGITS = 3, REVERSE = True):

    # Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
    # int is DIGITS.
    MAXLEN = DIGITS + 1 + DIGITS

    questions = []
    answers = []
    seen = set()
    print('Generating data...')
    while len(questions) < TRAINING_SIZE:
        f = lambda: int(''.join(np.random.choice(list('0123456789'))
                        for i in range(np.random.randint(1, DIGITS + 1))))
        a, b = f(), f()
        # Skip any addition questions we've already seen
        # Also skip any such that x+Y == Y+x (hence the sorting).
        key = tuple(sorted((a, b)))
        if key in seen:
            continue
        seen.add(key)
        # Pad the data with spaces such that it is always MAXLEN.
        q = '{}+{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a + b)
        # Answers can be of maximum size DIGITS + 1.
        ans += ' ' * (DIGITS + 1 - len(ans))
        if REVERSE:
            # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
            # space used for padding.)
            query = query[::-1]
        questions.append(query)
        answers.append(ans)
    print('Total addition questions:', len(questions))
    out = np.vstack((np.array(questions), np.array(answers)))
    return out.T

In [ ]:
train3char = generate_training(TRAINING_SIZE=50000, DIGITS=3, REVERSE=False)

In [ ]:
train3char.shape

In [ ]:
train3char

In [ ]:
r = np.random.randint(low=0, high=train3char.shape[0], size=4)
train3char[r, :]

In [ ]:
def vectorization(train, DIGITS):
    MAXLEN = 2*DIGITS+1
    print('Vectorization...')
    x = np.zeros((train.shape[0], MAXLEN, len(chars)), dtype=np.bool)
    y = np.zeros((train.shape[0], DIGITS + 1, len(chars)), dtype=np.bool)
    # encode questions
    for i, sentence in enumerate(train[:, 0]):
        x[i] = ctable.encode(sentence, MAXLEN)
    # encode answers
    for i, sentence in enumerate(train[:, 1]):
        y[i] = ctable.encode(sentence, DIGITS + 1)

    # Shuffle (x, y) in unison as the later parts of x will almost all be larger
    # digits.
    indices = np.arange(len(y))
    np.random.shuffle(indices)
    x = x[indices]
    y = y[indices]

    # Explicitly set apart 10% for validation data that we never train over.
    split_at = len(x) - len(x) // 10
    (x_train, x_val) = x[:split_at], x[split_at:]
    (y_train, y_val) = y[:split_at], y[split_at:]

    print('Shapes in training Data:')
    print(x_train.shape)
    print(y_train.shape)

    print('Shapes in validation Data:')
    print(x_val.shape)
    print(y_val.shape)
    return x_train, y_train, x_val, y_val

In [ ]:
x_train_3char, y_train_3char, x_val_3char, y_val_3char = vectorization(train=train3char, DIGITS=3)

In [ ]:
def build_model(chars, rnn_type='gru', DIGITS=3, HIDDEN_SIZE=10, BATCH_SIZE=100, DECODER_LAYERS=1):
    # ...
    return model

In [ ]:
model3char = build_model(chars=chars, rnn_type='lstm', DIGITS=3, HIDDEN_SIZE=128, BATCH_SIZE=128, DECODER_LAYERS=1)

In [ ]:
def train(model, x_train, y_train, x_val, y_val,  n_epochs=20, BATCH_SIZE=128, REVERSE=False):
    # Train the model each generation and show predictions against the validation
    # dataset.
    for iteration in range(1, n_epochs+1):
        # Train
        # ...
        
        # Select 5 samples from the validation set at random so we can visualize
        # errors.
        for i in range(5):
            # ...
            print('Question: ', #....
                 )
            print('Answer: ', # ...
                 )
            if correct == guess:
                print(colors.ok + '☑' + colors.close, end=' ')
            else:
                print(colors.fail + '☒' + colors.close, end=' ')
            print('Guess: ', guess, end='\n')

In [ ]:
train(model3char, x_train_3char, y_train_3char, x_val_3char, y_val_3char,  n_epochs=100, 
      BATCH_SIZE=128, REVERSE=False)

Check Five digits reversed:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs